# Sentiment Prediction in IMDB Reviews using an LSTM

In [1]:
import tempfile
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as fn
import fastestimator as fe
from fastestimator.dataset.data import imdb_review
from fastestimator.op.numpyop.univariate.reshape import Reshape
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.backend import load_model

In [2]:
MAX_WORDS = 10000
MAX_LEN = 500
batch_size = 64
epochs = 10
max_train_steps_per_epoch = None
max_eval_steps_per_epoch = None

In [3]:
# Parameters
epochs = 2
batch_size = 8
max_train_steps_per_epoch = 10
max_eval_steps_per_epoch = 10


<h2>Building components</h2>

### Step 1: Prepare training & evaluation data and define a `Pipeline`

We are loading the dataset from tf.keras.datasets.imdb which contains movie reviews and sentiment scores. All the words have been replaced with the integers that specifies the popularity of the word in corpus. To ensure all the sequences are of same length we need to pad the input sequences before defining the `Pipeline`.

In [4]:
train_data, eval_data = imdb_review.load_data(MAX_LEN, MAX_WORDS)
pipeline = fe.Pipeline(train_data=train_data,
                       eval_data=eval_data,
                       batch_size=batch_size,
                       ops=Reshape(1, inputs="y", outputs="y"))

    8192/17464789 [..............................] - ETA: 6:45

   24576/17464789 [..............................] - ETA: 3:50

   32768/17464789 [..............................] - ETA: 3:49

   49152/17464789 [..............................] - ETA: 3:05

   65536/17464789 [..............................] - ETA: 2:32

...

15269888/17464789 [=========================>....] - ETA: 0s

15712256/17464789 [=========================>....] - ETA: 0s

16171008/17464789 [==========================>...] - ETA: 0s

16629760/17464789 [===========================>..] - ETA: 0s

17465344/17464789 [==============================] - 3s 0us/step


### Step 2: Create a `model` and FastEstimator `Network`

First, we have to define the neural network architecture, and then pass the definition, associated model name, and optimizer into fe.build:

In [5]:
class ReviewSentiment(nn.Module):
    def __init__(self, embedding_size=64, hidden_units=64):
        super().__init__()
        self.embedding = nn.Embedding(MAX_WORDS, embedding_size)
        self.conv1d = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.maxpool1d = nn.MaxPool1d(kernel_size=4)
        self.lstm = nn.LSTM(input_size=125, hidden_size=hidden_units, num_layers=1)
        self.fc1 = nn.Linear(in_features=hidden_units, out_features=250)
        self.fc2 = nn.Linear(in_features=250, out_features=1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute((0, 2, 1))
        x = self.conv1d(x)
        x = fn.relu(x)
        x = self.maxpool1d(x)
        output, _ = self.lstm(x)
        x = output[:, -1]  # sequence output of only last timestamp
        x = fn.tanh(x)
        x = self.fc1(x)
        x = fn.relu(x)
        x = self.fc2(x)
        x = fn.sigmoid(x)
        return x

`Network` is the object that defines the whole training graph, including models, loss functions, optimizers etc. A `Network` can have several different models and loss functions (ex. GANs). `fe.Network` takes a series of operators, in this case just the basic `ModelOp`, loss op, and `UpdateOp` will suffice. It should be noted that "y_pred" is the key in the data dictionary which will store the predictions.

In [6]:
model = fe.build(model_fn=lambda: ReviewSentiment(), optimizer_fn="adam")
network = fe.Network(ops=[
    ModelOp(model=model, inputs="x", outputs="y_pred"),
    CrossEntropy(inputs=("y_pred", "y"), outputs="loss"),
    UpdateOp(model=model, loss_name="loss")
])

### Step 3: Prepare `Estimator` and configure the training loop

`Estimator` is the API that wraps the `Pipeline`, `Network` and other training metadata together. `Estimator` also contains `Traces`, which are similar to the callbacks of Keras.

In the training loop, we want to measure the validation loss and save the model that has the minimum loss. `BestModelSaver` is a convenient `Trace` to achieve this. Let's also measure accuracy over time using another `Trace`:

In [7]:
model_dir = tempfile.mkdtemp()
traces = [Accuracy(true_key="y", pred_key="y_pred"), BestModelSaver(model=model, save_dir=model_dir)]
estimator = fe.Estimator(network=network,
                         pipeline=pipeline,
                         epochs=epochs,
                         traces=traces,
                         max_train_steps_per_epoch=max_train_steps_per_epoch,
                         max_eval_steps_per_epoch=max_eval_steps_per_epoch)

<h2>Training</h2>

In [8]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        



FastEstimator-Start: step: 1; num_device: 0; logging_interval: 100; 


/home/geez219/anaconda3/envs/tf2.1/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/geez219/anaconda3/envs/tf2.1/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


FastEstimator-Train: step: 1; loss: 0.6929367; 


FastEstimator-Train: step: 10; epoch: 1; epoch_time: 0.24 sec; 


FastEstimator-BestModelSaver: Saved model to /tmp/tmp2gnn_jke/model_best_loss.pt
FastEstimator-Eval: step: 10; epoch: 1; loss: 0.6921011; accuracy: 0.525; since_best_loss: 0; min_loss: 0.6921011; 


FastEstimator-Train: step: 20; epoch: 2; epoch_time: 0.24 sec; 


FastEstimator-BestModelSaver: Saved model to /tmp/tmp2gnn_jke/model_best_loss.pt
FastEstimator-Eval: step: 20; epoch: 2; loss: 0.6915754; accuracy: 0.5625; since_best_loss: 0; min_loss: 0.6915754; 
FastEstimator-Finish: step: 20; total_time: 1.17 sec; model_lr: 0.001; 


<h2>Inferencing</h2>

For inferencing, first we have to load the trained model weights. We previously saved model weights corresponding to our minimum loss, and now we will load the weights using `load_model()`:

In [9]:
model_name = 'model_best_loss.pt'
model_path = os.path.join(model_dir, model_name)
load_model(model, model_path)

Let's get some random sequence and compare the prediction with the ground truth:

In [10]:
selected_idx = np.random.randint(10000)
print("Ground truth is: ",eval_data[selected_idx]['y'])

Ground truth is:  0


Create data dictionary for the inference. The `Transform()` function in Pipeline and Network applies all the operations on the given data:

In [11]:
infer_data = {"x":eval_data[selected_idx]['x'], "y":eval_data[selected_idx]['y']}
data = pipeline.transform(infer_data, mode="infer")
data = network.transform(data, mode="infer")

Finally, print the inferencing results.

In [12]:
print("Prediction for the input sequence: ", np.array(data["y_pred"])[0][0])

Prediction for the input sequence:  0.5036131
